In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Path to the training images
train_images_dir = '/kaggle/input/stanford-car-dataset-by-classes-folder/car_data/car_data/train'
image_size=224

train_datagen = ImageDataGenerator(
    rescale=1./255, 
    validation_split=0.2, 
    shear_range=0.2,
    zoom_range=0.2,  
    horizontal_flip=True,  
    brightness_range=[0.8, 1.2], 
    rotation_range=15,  
    width_shift_range=0.2,  
    height_shift_range=0.2, 
    fill_mode='nearest',  
    channel_shift_range=5.0,  
)


# Create the training generator for training data
train_generator = train_datagen.flow_from_directory(
    train_images_dir,  # Path to the training images
    target_size=(image_size, image_size),  # Resize images to target size (set image_size beforehand)
    batch_size=32,  # Number of images to process in each batch
    class_mode='categorical',  # For multi-class classification
    subset='training',  # Set subset to 'training' for training data
        shuffle=True,  # Shuffle data after each epoch
)

# Create the validation generator
validation_generator = train_datagen.flow_from_directory(
    train_images_dir,  # Path to the training images
    target_size=(image_size, image_size),  # Resize images to target size (set image_size beforehand)
    batch_size=32,  # Number of images to process in each batch
    class_mode='categorical',  # For multi-class classification
    subset='validation',  # Set subset to 'validation' for validation data
    shuffle=False,  # Don't shuffle validation data
)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks
import os

# Required constants
image_size = 224  # Image size (224x224 is the default input size for DenseNet)
batch_size = 32
epochs = 55
num_classes = 196  # Number of classes for classification (replace with actual number of classes)
learning_rate = 0.01

# Load pre-trained DenseNet121 model (without the top layers)
base_model = tf.keras.applications.DenseNet121(
    include_top=False,  # Exclude the fully connected layers at the top
    weights='imagenet',  # Use ImageNet weights
    input_shape=(image_size, image_size, 3)  # Input size (224x224x3)
)

# Freeze the layers in DenseNet to avoid retraining them
base_model.trainable = False

# Build the full model with custom classification head
model = models.Sequential([
    base_model,  # Add DenseNet backbone as base
    layers.GlobalAveragePooling2D(),  # Global average pooling to reduce the spatial dimensions
    layers.Dense(4096, activation='relu'),  # Dense layer (1024 units)
    #layers.Dense(320, activation='relu'),  # Dense layer (320 units)
    layers.Dropout(0.5),  # Dropout with 50% probability
    layers.Dense(num_classes, activation='softmax')  # Final classification layer (output layer)
])
optimizer = optimizers.Adam(learning_rate=0.001)  # Default learning rate for Adam

for layer in base_model.layers[-20:]:
    layer.trainable = True
    
model.compile(optimizer=optimizer, 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

history = model.fit(
    train_generator,  # Assuming you have 'train_generator' from ImageDataGenerator
    epochs=epochs,
    validation_data=validation_generator,  # Assuming you have 'validation_generator'
    steps_per_epoch=train_generator.samples // batch_size,  # Number of batches per epoch
    validation_steps=validation_generator.samples // batch_size,  # Number of validation steps
    #callbacks=[ checkpoint_callback]  # Include the callbacks
)

In [ ]:
test_dir = '/kaggle/input/stanford-car-dataset-by-classes-folder/car_data/car_data/test'
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='categorical',  # For multi-class classification
    shuffle=False  # Don't shuffle test data
)

In [ ]:
# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_acc}')


In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 4))

# Accuracy Curve
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Loss Curve
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()


In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Get Ground Truth Labels and Predictions
Y_true = validation_generator.classes  # Ground truth
Y_pred = model.predict(validation_generator)
Y_pred_classes = np.argmax(Y_pred, axis=1)

# Compute Confusion Matrix
cm = confusion_matrix(Y_true, Y_pred_classes)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=validation_generator.class_indices.keys())

# Visualize Confusion Matrix
plt.figure(figsize=(8, 6))
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()


In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Get Ground Truth Labels and Predictions
Y_true = validation_generator.classes  # Ground truth
Y_pred = model.predict(validation_generator)
Y_pred_classes = np.argmax(Y_pred, axis=1)

# Compute Confusion Matrix
cm = confusion_matrix(Y_true, Y_pred_classes)

# Sum of misclassifications per class
misclassifications = np.sum(cm, axis=1) - np.diag(cm)

# Select Top-K Classes with Most Misclassifications
K = 10
top_k_classes = np.argsort(misclassifications)[-K:]

# Create Reduced Confusion Matrix
cm_reduced = cm[np.ix_(top_k_classes, top_k_classes)]
class_labels = np.array(list(validation_generator.class_indices.keys()))[top_k_classes]

# Display Reduced Confusion Matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm_reduced, display_labels=class_labels)

plt.figure(figsize=(8, 6))
disp.plot(cmap=plt.cm.Blues, xticks_rotation=45)
plt.title(f"Top-{K} Confusion Matrix")
plt.show()


In [ ]:
from sklearn.metrics import classification_report

# Generate Classification Report
print("Classification Report:\n")
report = classification_report(Y_true, Y_pred_classes, target_names=validation_generator.class_indices.keys())
print(report)


In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize

# Binarize the labels for multi-class ROC
n_classes = len(validation_generator.class_indices)
Y_true_bin = label_binarize(Y_true, classes=[i for i in range(n_classes)])

# Compute ROC curve and AUC for each class
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(Y_true_bin[:, i], Y_pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot ROC curves
plt.figure(figsize=(10, 7))
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i], label=f"Class {i} (AUC = {roc_auc[i]:.2f})")

plt.plot([0, 1], [0, 1], 'k--', label='Random Guessing')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curves for Multi-Class Classification")
plt.legend(loc="lower right")
plt.show()
